In [2]:
# Import libraries
from calendar import monthrange
from datetime import datetime, timedelta
from pandas import DataFrame, Series
import pandas as pd
import numpy as np
import re

In [3]:
# global constants
path_to_data = "./bbdata"

In [4]:
# Loads a month of data, given year and month number
# TODO: document
def load_month_trip_data(year, month):
    trips = pd.read_csv(f"{path_to_data}/trips/{year}{month:02d}.csv",
                      parse_dates=["starttime", "stoptime"])
    
    # Drop unnecessary columns
    trips.drop("start station name", axis=1, inplace=True)
    trips.drop("end station name", axis=1, inplace=True)
    trips.drop("bikeid", axis=1, inplace=True)
    trips.drop("usertype", axis=1, inplace=True)
    trips.drop("birth year", axis=1, inplace=True)
    trips.drop("gender", axis=1, inplace=True)
    
    trips['starttime'] = pd.to_datetime(trips['starttime'])
    trips['stoptime'] = pd.to_datetime(trips['stoptime'])
    
    trips['start station latitude'] = pd.to_numeric(trips['start station latitude'])
    trips['start station longitude'] = pd.to_numeric(trips['start station longitude'])
    return trips

In [5]:
# Loads trip data from start_year/start_month to end_year/end_month
def load_trip_date(start_year, start_month, end_year, end_month):
    trips = DataFrame()
    for year in range(start_year, end_year + 1):
        for month in range(start_month, 13 if start_year is not end_year else end_month + 1):
            trips.append(load_month_trip_data(year, month))
        start_month = 1
    return trips

In [6]:
# Loads station data
# TODO: document
def load_station_data():
    stations = pd.read_csv(f"{path_to_data}/station_data.csv")
    stations.drop("Station", axis=1, inplace=True)
    return stations

In [7]:
# Loads weather data
# TODO: document
all_weather = pd.read_csv(f"{path_to_data}/weather_data_daily.csv")

all_weather.drop(["STATION","NAME","LATITUDE","LONGITUDE","ELEVATION"], axis = 1, inplace = True)    
all_weather[['YEAR','MONTH','DAY']] = all_weather['DATE'].str.split('-',expand=True)
all_weather.drop(["DATE"], axis = 1, inplace = True)

all_weather["YEAR"] = pd.to_numeric(all_weather["YEAR"])
all_weather["DAY"] = pd.to_numeric(all_weather["DAY"])
all_weather["MONTH"] = pd.to_numeric(all_weather["MONTH"])
all_weather["PRCP"] = pd.to_numeric(all_weather["PRCP"])
all_weather["SNOW"] = pd.to_numeric(all_weather["SNOW"])
all_weather["TMIN"] = pd.to_numeric(all_weather["TMIN"])
all_weather["TMAX"] = pd.to_numeric(all_weather["TMAX"])
all_weather["TAVG"] = pd.to_numeric(all_weather["TAVG"])

all_weather.apply(lambda row: ((row['TMIN'] * row['TMAX']) / 2) if np.isnan(row['TAVG']) else row['TAVG'], 
                  axis = 1)

# Loads weather data
# TODO: document
def load_weather_data(day, month, year):
    weather = all_weather.loc[(all_weather['YEAR'] == year) \
                              & (all_weather['DAY'] == day) \
                              & (all_weather['MONTH'] == month)].copy()
    weather.drop(['YEAR','MONTH','DAY'], axis = 1, inplace = True)
    weather.reset_index()
    return dict(weather.iloc[0])

In [8]:
# TODO: document
def get_starting_trips_in_range(trips, start, end):
    mask = (trips['starttime'] > start) & (trips['starttime'] <= end)
    return trips.loc[mask]

# TODO: document
def get_ending_trips_in_range(trips, start, end):
    
    mask = (trips['stoptime'] > start) & (trips['stoptime'] <= end)
    return trips.loc[mask]

In [13]:
# Process trip data into time series with availability value for each staation
# TODO: document
# time_step_delta should evenly divide minutes in a day (60 * 24 = 1440)
def trips_to_availability_time_series(start_year, start_month, 
                                      end_year, end_month, 
                                      time_step_delta, 
                                      export_url,
                                      verbose_level=0):
    if verbose_level >= 0:
        print("Starting time series generation")
    # TODO: offload to file after n steps?
    stations = load_station_data()
    
    step_delta = timedelta(minutes=time_step_delta)
    
    end_time = datetime(end_year, end_month, monthrange(end_year, end_month)[1], 23, 59)
    
    current_time = datetime(start_year, start_month, 1, 0, 0)
    current_month_trip_data = load_month_trip_data(current_time.year, current_time.month)
    
    if verbose_level >= 1:
        print(f"Starting with ({current_time.strftime('%b')} {current_time.year})")
    
    current_day_weather_data = load_weather_data(current_time.day, current_time.month, current_time.year)
    if verbose_level >= 3:
        print(f"\t\tLoaded Weather data for {current_time.strftime('%b')} {current_time.day} - {current_time.year}")
    
    time_series = [] # use to compute current available bikes

    while current_time <= end_time:
        next_time = current_time + step_delta
        
        current_starting_trips = get_starting_trips_in_range(current_month_trip_data, current_time, next_time)
        current_ending_trips = get_ending_trips_in_range(current_month_trip_data, current_time, next_time)
        
        for index, station in stations.iterrows():
            mask_from = current_starting_trips["start station latitude"] == station["Latitude"]
            mask_to = current_ending_trips["end station latitude"] == station["Latitude"]
            trips_from_station = current_starting_trips.loc[mask_from].shape[0]
            trips_to_station = current_ending_trips.loc[mask_to].shape[0]
                
            station_time_series_entry = {
                "time_step_start": current_time, 
                "time_step_end": next_time, 
                "station": station["Station ID"], 
                "latitude": station["Latitude"], 
                "longitude": station["Longitude"], 
                "municipality": station["Municipality"], 
                "total_docks": station["# of Docks"], 
                "trips_from_station": trips_from_station,
                "trips_to_station": trips_to_station, 
                "PRCP": current_day_weather_data["PRCP"],
                "SNOW": current_day_weather_data["SNOW"],
                "TMIN": current_day_weather_data["TMIN"],
                "TMAX": current_day_weather_data["TMAX"],
                "TAVG": current_day_weather_data["TAVG"]
            }
                        
            time_series.append(station_time_series_entry)
            
        if verbose_level >= 2:
            print(f"\tCompleted time step [{current_time} --- {next_time}]; Current rows: {len(time_series)}")
            
        if current_time.month is not next_time.month:
            # Export to csv
            time_series_df = DataFrame(time_series)
            with open(export_url, 'a') as file:
                time_series_df.to_csv(file, header = file.tell() == 0, index = False, line_terminator = '\n')
            if verbose_level >= 3:
                print("Exported to file.")
            time_series = []
            
            # Next month
            current_month_trip_data = load_month_trip_data(next_time.year, next_time.month)
            if verbose_level >= 1:
                print(f"Moving to next month ({next_time.strftime('%b')} {next_time.year})")
                
        if current_time.day is not next_time.day:
            current_day_weather_data = load_weather_data(next_time.day, next_time.month, next_time.year)
            if verbose_level >= 3:
                print(f"\t\tLoaded Weather data for {next_time.strftime('%b')} {next_time.day} - {next_time.year}")
        
        current_time = next_time
        
    if verbose_level >= 0:
        print("Completed full date range")

In [14]:
#trips_to_availability_time_series(2019, 1, 2019, 10, 60 * 24 * 30, "./timeseries.monthly.csv",        verbose_level = 0)
#trips_to_availability_time_series(2019, 1, 2019, 10, 60 * 24 * 7,  "./timeseries.weekly.csv",         verbose_level = 0)
#trips_to_availability_time_series(2019, 1, 2019, 10, 60 * 24,      "./timeseries.daily.csv",          verbose_level = 0)
#trips_to_availability_time_series(2019, 1, 2019, 10, 60 * 12,      "./timeseries.12-hour-ly.csv",     verbose_level = 0)
#trips_to_availability_time_series(2019, 1, 2019, 10, 60 * 6,       "./timeseries.6-hour-ly.csv",      verbose_level = 0)
trips_to_availability_time_series(2019, 1, 2019, 9, 60,           "./timeseries.hourly.2019.csv",         verbose_level = 3)
trips_to_availability_time_series(2019, 1, 2019, 9, 60 * 24 * 30, "./timeseries.monthly.2019.csv",        verbose_level = 0)
trips_to_availability_time_series(2019, 1, 2019, 9, 60 * 24 * 7,  "./timeseries.weekly.2019.csv",         verbose_level = 0)
trips_to_availability_time_series(2019, 1, 2019, 9, 60 * 24,      "./timeseries.daily.2019.csv",          verbose_level = 0)
#trips_to_availability_time_series(2019, 1, 2019, 10, 30,           "./timeseries.half-hourly.csv",    verbose_level = 0)
#trips_to_availability_time_series(2019, 1, 2019, 10, 15,           "./timeseries.quarter-hourly.csv", verbose_level = 0)
#trips_to_availability_time_series(2019, 1, 2019, 10, 5,            "./timeseries.5-minute-ly.csv",    verbose_level = 0)

Starting time series generation
Starting with (Jan 2019)
		Loaded Weather data for Jan 1 - 2019
	Completed time step [2019-01-01 00:00:00 --- 2019-01-01 01:00:00]; Current rows: 281
	Completed time step [2019-01-01 01:00:00 --- 2019-01-01 02:00:00]; Current rows: 562
	Completed time step [2019-01-01 02:00:00 --- 2019-01-01 03:00:00]; Current rows: 843
	Completed time step [2019-01-01 03:00:00 --- 2019-01-01 04:00:00]; Current rows: 1124
	Completed time step [2019-01-01 04:00:00 --- 2019-01-01 05:00:00]; Current rows: 1405
	Completed time step [2019-01-01 05:00:00 --- 2019-01-01 06:00:00]; Current rows: 1686
	Completed time step [2019-01-01 06:00:00 --- 2019-01-01 07:00:00]; Current rows: 1967
	Completed time step [2019-01-01 07:00:00 --- 2019-01-01 08:00:00]; Current rows: 2248
	Completed time step [2019-01-01 08:00:00 --- 2019-01-01 09:00:00]; Current rows: 2529
	Completed time step [2019-01-01 09:00:00 --- 2019-01-01 10:00:00]; Current rows: 2810
	Completed time step [2019-01-01 10:0

	Completed time step [2019-01-04 20:00:00 --- 2019-01-04 21:00:00]; Current rows: 26133
	Completed time step [2019-01-04 21:00:00 --- 2019-01-04 22:00:00]; Current rows: 26414
	Completed time step [2019-01-04 22:00:00 --- 2019-01-04 23:00:00]; Current rows: 26695
	Completed time step [2019-01-04 23:00:00 --- 2019-01-05 00:00:00]; Current rows: 26976
		Loaded Weather data for Jan 5 - 2019
	Completed time step [2019-01-05 00:00:00 --- 2019-01-05 01:00:00]; Current rows: 27257
	Completed time step [2019-01-05 01:00:00 --- 2019-01-05 02:00:00]; Current rows: 27538
	Completed time step [2019-01-05 02:00:00 --- 2019-01-05 03:00:00]; Current rows: 27819
	Completed time step [2019-01-05 03:00:00 --- 2019-01-05 04:00:00]; Current rows: 28100
	Completed time step [2019-01-05 04:00:00 --- 2019-01-05 05:00:00]; Current rows: 28381
	Completed time step [2019-01-05 05:00:00 --- 2019-01-05 06:00:00]; Current rows: 28662
	Completed time step [2019-01-05 06:00:00 --- 2019-01-05 07:00:00]; Current rows:

	Completed time step [2019-01-08 16:00:00 --- 2019-01-08 17:00:00]; Current rows: 51985
	Completed time step [2019-01-08 17:00:00 --- 2019-01-08 18:00:00]; Current rows: 52266
	Completed time step [2019-01-08 18:00:00 --- 2019-01-08 19:00:00]; Current rows: 52547
	Completed time step [2019-01-08 19:00:00 --- 2019-01-08 20:00:00]; Current rows: 52828
	Completed time step [2019-01-08 20:00:00 --- 2019-01-08 21:00:00]; Current rows: 53109
	Completed time step [2019-01-08 21:00:00 --- 2019-01-08 22:00:00]; Current rows: 53390
	Completed time step [2019-01-08 22:00:00 --- 2019-01-08 23:00:00]; Current rows: 53671
	Completed time step [2019-01-08 23:00:00 --- 2019-01-09 00:00:00]; Current rows: 53952
		Loaded Weather data for Jan 9 - 2019
	Completed time step [2019-01-09 00:00:00 --- 2019-01-09 01:00:00]; Current rows: 54233
	Completed time step [2019-01-09 01:00:00 --- 2019-01-09 02:00:00]; Current rows: 54514
	Completed time step [2019-01-09 02:00:00 --- 2019-01-09 03:00:00]; Current rows:

	Completed time step [2019-01-12 12:00:00 --- 2019-01-12 13:00:00]; Current rows: 77837
	Completed time step [2019-01-12 13:00:00 --- 2019-01-12 14:00:00]; Current rows: 78118
	Completed time step [2019-01-12 14:00:00 --- 2019-01-12 15:00:00]; Current rows: 78399
	Completed time step [2019-01-12 15:00:00 --- 2019-01-12 16:00:00]; Current rows: 78680
	Completed time step [2019-01-12 16:00:00 --- 2019-01-12 17:00:00]; Current rows: 78961
	Completed time step [2019-01-12 17:00:00 --- 2019-01-12 18:00:00]; Current rows: 79242
	Completed time step [2019-01-12 18:00:00 --- 2019-01-12 19:00:00]; Current rows: 79523
	Completed time step [2019-01-12 19:00:00 --- 2019-01-12 20:00:00]; Current rows: 79804
	Completed time step [2019-01-12 20:00:00 --- 2019-01-12 21:00:00]; Current rows: 80085
	Completed time step [2019-01-12 21:00:00 --- 2019-01-12 22:00:00]; Current rows: 80366
	Completed time step [2019-01-12 22:00:00 --- 2019-01-12 23:00:00]; Current rows: 80647
	Completed time step [2019-01-12

	Completed time step [2019-01-16 08:00:00 --- 2019-01-16 09:00:00]; Current rows: 103689
	Completed time step [2019-01-16 09:00:00 --- 2019-01-16 10:00:00]; Current rows: 103970
	Completed time step [2019-01-16 10:00:00 --- 2019-01-16 11:00:00]; Current rows: 104251
	Completed time step [2019-01-16 11:00:00 --- 2019-01-16 12:00:00]; Current rows: 104532
	Completed time step [2019-01-16 12:00:00 --- 2019-01-16 13:00:00]; Current rows: 104813
	Completed time step [2019-01-16 13:00:00 --- 2019-01-16 14:00:00]; Current rows: 105094
	Completed time step [2019-01-16 14:00:00 --- 2019-01-16 15:00:00]; Current rows: 105375
	Completed time step [2019-01-16 15:00:00 --- 2019-01-16 16:00:00]; Current rows: 105656
	Completed time step [2019-01-16 16:00:00 --- 2019-01-16 17:00:00]; Current rows: 105937
	Completed time step [2019-01-16 17:00:00 --- 2019-01-16 18:00:00]; Current rows: 106218
	Completed time step [2019-01-16 18:00:00 --- 2019-01-16 19:00:00]; Current rows: 106499
	Completed time step 

...

	Completed time step [2019-09-13 08:00:00 --- 2019-09-13 09:00:00]; Current rows: 83457
	Completed time step [2019-09-13 09:00:00 --- 2019-09-13 10:00:00]; Current rows: 83738
	Completed time step [2019-09-13 10:00:00 --- 2019-09-13 11:00:00]; Current rows: 84019
	Completed time step [2019-09-13 11:00:00 --- 2019-09-13 12:00:00]; Current rows: 84300
	Completed time step [2019-09-13 12:00:00 --- 2019-09-13 13:00:00]; Current rows: 84581
	Completed time step [2019-09-13 13:00:00 --- 2019-09-13 14:00:00]; Current rows: 84862
	Completed time step [2019-09-13 14:00:00 --- 2019-09-13 15:00:00]; Current rows: 85143
	Completed time step [2019-09-13 15:00:00 --- 2019-09-13 16:00:00]; Current rows: 85424
	Completed time step [2019-09-13 16:00:00 --- 2019-09-13 17:00:00]; Current rows: 85705
	Completed time step [2019-09-13 17:00:00 --- 2019-09-13 18:00:00]; Current rows: 85986
	Completed time step [2019-09-13 18:00:00 --- 2019-09-13 19:00:00]; Current rows: 86267
	Completed time step [2019-09-13

	Completed time step [2019-09-17 03:00:00 --- 2019-09-17 04:00:00]; Current rows: 109028
	Completed time step [2019-09-17 04:00:00 --- 2019-09-17 05:00:00]; Current rows: 109309
	Completed time step [2019-09-17 05:00:00 --- 2019-09-17 06:00:00]; Current rows: 109590
	Completed time step [2019-09-17 06:00:00 --- 2019-09-17 07:00:00]; Current rows: 109871
	Completed time step [2019-09-17 07:00:00 --- 2019-09-17 08:00:00]; Current rows: 110152
	Completed time step [2019-09-17 08:00:00 --- 2019-09-17 09:00:00]; Current rows: 110433
	Completed time step [2019-09-17 09:00:00 --- 2019-09-17 10:00:00]; Current rows: 110714
	Completed time step [2019-09-17 10:00:00 --- 2019-09-17 11:00:00]; Current rows: 110995
	Completed time step [2019-09-17 11:00:00 --- 2019-09-17 12:00:00]; Current rows: 111276
	Completed time step [2019-09-17 12:00:00 --- 2019-09-17 13:00:00]; Current rows: 111557
	Completed time step [2019-09-17 13:00:00 --- 2019-09-17 14:00:00]; Current rows: 111838
	Completed time step 

	Completed time step [2019-09-20 22:00:00 --- 2019-09-20 23:00:00]; Current rows: 134599
	Completed time step [2019-09-20 23:00:00 --- 2019-09-21 00:00:00]; Current rows: 134880
		Loaded Weather data for Sep 21 - 2019
	Completed time step [2019-09-21 00:00:00 --- 2019-09-21 01:00:00]; Current rows: 135161
	Completed time step [2019-09-21 01:00:00 --- 2019-09-21 02:00:00]; Current rows: 135442
	Completed time step [2019-09-21 02:00:00 --- 2019-09-21 03:00:00]; Current rows: 135723
	Completed time step [2019-09-21 03:00:00 --- 2019-09-21 04:00:00]; Current rows: 136004
	Completed time step [2019-09-21 04:00:00 --- 2019-09-21 05:00:00]; Current rows: 136285
	Completed time step [2019-09-21 05:00:00 --- 2019-09-21 06:00:00]; Current rows: 136566
	Completed time step [2019-09-21 06:00:00 --- 2019-09-21 07:00:00]; Current rows: 136847
	Completed time step [2019-09-21 07:00:00 --- 2019-09-21 08:00:00]; Current rows: 137128
	Completed time step [2019-09-21 08:00:00 --- 2019-09-21 09:00:00]; Cu

	Completed time step [2019-09-24 17:00:00 --- 2019-09-24 18:00:00]; Current rows: 160170
	Completed time step [2019-09-24 18:00:00 --- 2019-09-24 19:00:00]; Current rows: 160451
	Completed time step [2019-09-24 19:00:00 --- 2019-09-24 20:00:00]; Current rows: 160732
	Completed time step [2019-09-24 20:00:00 --- 2019-09-24 21:00:00]; Current rows: 161013
	Completed time step [2019-09-24 21:00:00 --- 2019-09-24 22:00:00]; Current rows: 161294
	Completed time step [2019-09-24 22:00:00 --- 2019-09-24 23:00:00]; Current rows: 161575
	Completed time step [2019-09-24 23:00:00 --- 2019-09-25 00:00:00]; Current rows: 161856
		Loaded Weather data for Sep 25 - 2019
	Completed time step [2019-09-25 00:00:00 --- 2019-09-25 01:00:00]; Current rows: 162137
	Completed time step [2019-09-25 01:00:00 --- 2019-09-25 02:00:00]; Current rows: 162418
	Completed time step [2019-09-25 02:00:00 --- 2019-09-25 03:00:00]; Current rows: 162699
	Completed time step [2019-09-25 03:00:00 --- 2019-09-25 04:00:00]; Cu

	Completed time step [2019-09-28 12:00:00 --- 2019-09-28 13:00:00]; Current rows: 185741
	Completed time step [2019-09-28 13:00:00 --- 2019-09-28 14:00:00]; Current rows: 186022
	Completed time step [2019-09-28 14:00:00 --- 2019-09-28 15:00:00]; Current rows: 186303
	Completed time step [2019-09-28 15:00:00 --- 2019-09-28 16:00:00]; Current rows: 186584
	Completed time step [2019-09-28 16:00:00 --- 2019-09-28 17:00:00]; Current rows: 186865
	Completed time step [2019-09-28 17:00:00 --- 2019-09-28 18:00:00]; Current rows: 187146
	Completed time step [2019-09-28 18:00:00 --- 2019-09-28 19:00:00]; Current rows: 187427
	Completed time step [2019-09-28 19:00:00 --- 2019-09-28 20:00:00]; Current rows: 187708
	Completed time step [2019-09-28 20:00:00 --- 2019-09-28 21:00:00]; Current rows: 187989
	Completed time step [2019-09-28 21:00:00 --- 2019-09-28 22:00:00]; Current rows: 188270
	Completed time step [2019-09-28 22:00:00 --- 2019-09-28 23:00:00]; Current rows: 188551
	Completed time step 